Có thể dùng clustering gom cụm

# Chuẩn bị dữ liệu

In [1]:
import pyodbc
import pandas as pd
import numpy as np

In [2]:
server_name = 'DESKTOP-TOL1F2F\MSSQLSERVERBI'
database_name = 'AdventureWorksDW2012'

connection = pyodbc.connect('Driver={SQL Server}; Server=' + server_name +
                      '; Database=' + database_name +
                      '; Trusted_Connection=yes;')

In [3]:
df = pd.read_sql_query(
    "select c.Gender as gender, \
        c.YearlyIncome as yearly_income, \
        c.MaritalStatus as marital_status, \
        c.TotalChildren as total_children, \
        c.NumberChildrenAtHome as children_at_home, \
        c.HouseOwnerFlag as house_owner, \
        c.NumberCarsOwned as cars_owned, \
        c.EnglishOccupation as occupation, \
        c.EnglishEducation as education,\
        c.CommuteDistance as commute_distance \
        , case sum(case pc.EnglishProductCategoryName when 'Bikes' then 1 else 0 end) \
          when 0 then 0 else 1 end as bikes \
    from FactInternetSales f \
        join DimCustomer c on f.CustomerKey = c.CustomerKey \
        join DimProduct p on f.ProductKey = p.ProductKey \
        join DimProductSubcategory psc on p.ProductSubcategoryKey = psc.ProductSubcategoryKey \
        join DimProductCategory pc on psc.ProductCategoryKey = pc.ProductCategoryKey \
    group by c.Gender, c.YearlyIncome, c.MaritalStatus, c.TotalChildren, c.NumberChildrenAtHome, c.HouseOwnerFlag, \
        c.NumberCarsOwned, c.EnglishOccupation, c.EnglishEducation, c.CommuteDistance",
    connection) 

df.head()

,gender,yearly_income,marital_status,total_children,children_at_home,house_owner,cars_owned,occupation,education,commute_distance,bikes
0,M,90000.0,M,4,1,0,3,Management,High School,5-10 Miles,1
1,F,130000.0,M,1,3,1,3,Management,Bachelors,2-5 Miles,1
2,F,20000.0,M,2,1,1,2,Clerical,Partial High School,1-2 Miles,1
3,M,30000.0,M,1,0,1,1,Manual,High School,1-2 Miles,1
4,M,60000.0,M,3,2,0,2,Professional,Bachelors,0-1 Miles,1


# Thống kê mô tả
Tập dữ liệu này giống với tập dữ liệu được sử dụng ở Yêu cầu 2. Khác ở chỗ không có 3 cột thuộc tính phụ thuộc accessories, components, clothing.

# Bước dưới đây cứ chọn mô hình rồi làm thôi nha, view dữ liệu bên Yêu cầu 2 rồi